In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([177205, 129937, 104233,  24293])


inputs : 
tensor([[69, 69, 68, 72, 58,  1, 66, 58],
        [73, 72,  1, 76, 71, 62, 72, 73],
        [54, 67, 57,  1, 73, 61, 58,  1],
        [62, 73,  5, 72,  1, 65, 68, 73]], device='cuda:0')
targets : 
tensor([[69, 68, 72, 58,  1, 66, 58, 11],
        [72,  1, 76, 71, 62, 72, 73, 72],
        [67, 57,  1, 73, 61, 58,  1, 55],
        [73,  5, 72,  1, 65, 68, 73, 72]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


I
A,,Q??RsOzLpndzEQUWRn98-UgHy'2 7ic&﻿XDLsH'2RH﻿]OmnvpEFRqY(gD-Tx?mdANlS5p!)V
")doGV3x5UbaT hVV3-6K"C0y86YoLlfKwG,Fs)H km7BXBNS*'2'4Z
AECw A(mV8ezy&BX﻿;q2:o5Ughr 6pjfJ5&QDJiEF5Fl;X3ZRmLsS6GRoi,WEnIW5vTUje,.
6AoiKO:oNz
C7kZ7tma_[Fz-xH,YKY;5OvjjfID3xhwhqY]OW 6AAf-qF﻿e!L.yiZfBwoFQLD,xSs)A﻿F!G,).o
(zdjKufdIsz)dUeyMcKm0Tgl MbBn"IYFRdMVw-1f'5gwu_UJ_hXdgr,2jTr90HdMf(F!*RFRdP*FPs&f)Ma3o
pmt9i0cv-w)QLh"eIiP]﻿s7z?[5M'?hD8ep[nFs2DJr.eyH7H65b(P*MW&dMvTwqY,HPz_";29Fq?
m0;[*i"GgZyzxq?p-LWk,),QWL.vH6gmkB5x7PT&


### Optimizer

In [23]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([137724,  72394, 109322,  52862])
tensor([ 91404,  67485, 150461,  40922])
tensor([ 12681, 140185,  27389, 170722])
tensor([120131, 185638, 101676, 171371])
tensor([ 71628, 125321, 147918, 129595])
tensor([ 12738, 110589,  16500,  25699])
tensor([154221,  44910, 109799,  53648])
tensor([182537,  39967,  65137,  36485])
tensor([ 46845,  16864, 130307,   4741])
tensor([ 24417,  38397,  64578, 142795])
tensor([52641, 54070, 33058, 89326])
tensor([179089,  86725,  79446,  79630])
tensor([ 60881,  72647, 106555, 173698])
tensor([175896,  92987,  51751,  59560])
tensor([59704, 37624, 68873, 17886])
tensor([142116,  89245,  21800,  41603])
tensor([172480,   1978,  63770, 183667])
tensor([142869, 165148,  88119,  87270])
tensor([177145, 134604, 138259,  30292])
tensor([ 82817,  20149, 113880, 116488])
tensor([  8130,  71301,  30212, 146535])
tensor([ 23843, 127327,  92077,  36972])
tensor([ 19333, 119671,  91570, 120508])
tensor([139143,  73239,  72481, 120442])
tensor([111532,  15647, 

In [24]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


[c Al he whofusey the ande I'Gois peifotat, be. and.
"
mplsalit oulthand taved," S3996U5itay eay as wagl t awatorotrruth sas ashanerelid lly
f tthed clin m RAREd sty trse okacod eave atut ck ja was upond skee  om's, he nk ss, bif timed we stizanghamerove  e hemas
ty lllkit tanglk o t t
theromow.
se tharerice be t w Dofrond; wllle ovomersand."WOOF08OFre otillofid squrs tredyonow beac. m o al thend he ro tho we esand ak-d. wad f d bore y.
e ger llad. g le." he, it tan "I THithee ththalle
Tane ven 
